# Frobenius Monoids
###### (Seven Sketches in Compositionality Chapter 6)

A category is said to have a frobenius structure if it allows for both monoid and comonoid operations.
```
mu :: X + X -> X
eta :: 0 -> X
delta :: X -> X + X
epsilon :: X -> 0
```
The categories of Cospans and Spans are examples.

In [29]:
:ext ExistentialQuantification
:ext TypeFamilies
import Data.Bifunctor
import Data.Void
import Control.Category
import Control.Monad
import Prelude hiding ((.), id)

class Frobenius m where
  type Prod m :: * -> * -> *
  rmap :: m b c -> m (Prod m a b) (Prod m a c)
  lmap :: m a c -> m (Prod m a b) (Prod m c b)
  rassoc :: m (Prod m (Prod m a b) c) (Prod m a (Prod m b c))
  lassoc :: m (Prod m a (Prod m b c)) (Prod m (Prod m a b) c)
  
  type Zero m :: *
  mu :: m (Prod m x x) x
  eta :: m (Zero m) x
  delta :: m x (Prod m x x)
  epsilon :: m x (Zero m)

---

data Cospan a b = forall x. Cospan (a -> x) (b -> x)

instance Category Cospan where
  id = Cospan id id
  Cospan f g . Cospan u v = Cospan (Left . u) (Right . g)

instance Frobenius Cospan where
  type Prod Cospan = Either
  rmap (Cospan f g) = Cospan (second f) (second g)
  lmap (Cospan f g) = Cospan (first f) (first g)
  rassoc = Cospan (either (either Left (Right . Left)) (Right . Right))
                  id
  lassoc = Cospan id
                  (either (either Left (Right . Left)) (Right . Right))
                  
  type Zero Cospan = Void
  
  mu = Cospan (either id id) id
  eta = Cospan absurd id
  delta = Cospan id (either id id)
  epsilon = Cospan id absurd
  
---
  
data Span a b = forall x. Span (x -> a) (x -> b)

instance Category Span where
  id = Span id id
  Span f g . Span u v = Span (u . fst) (g . snd)
  
instance Frobenius Span where
  type Prod Span = (,)
  rmap (Span f g) = Span (second f) (second g)
  lmap (Span f g) = Span (first f) (first g)
  rassoc = Span (\(a,(b,c)) -> ((a,b),c)) id
  lassoc = Span id (\(a,(b,c)) -> ((a,b),c))
  
  type Zero Span = ()
  
  mu = Span (join (,)) id
  eta = Span (const ()) id
  delta = Span id (join (,))
  epsilon = Span id (const ())

A frobenius monoid must satisfy the following identities
```
frobenius equation
A x A ; A x (A x A) ; (A x A) x A ; A x A =
A x A ; (A x A) x A ; A x (A x A) ; A x A
special
mu . delta = id
```

In [33]:
fro :: (Category m, Frobenius m) => m (Prod m x x) (Prod m x x)
fro = lmap mu . lassoc . rmap delta
-- how to show these are equal?
orf :: (Category m, Frobenius m) => m (Prod m x x) (Prod m x x)
orf = rmap mu . rassoc . lmap delta

special :: (Category m, Frobenius m) => m x x
special = mu . delta